# Associating libclang callsites with declarations

How do you find the locations (line and column position) of all the references of a specific function declaration when parsing a C++ source file via libclang in Python.

In [34]:
!cat tmp.cpp

#include <iostream>
using namespace std;

namespace impl {
    int addition(int x, int y) {
        return x + y;
    }

    void f() {
        addition(2, 3);
    }
}

int addition (int a, int b) {
  int r;
  r=a+b;
  return r;
}

int main () {
  int z, q;
  z = addition (5,3);
  q = addition (5,5);
  cout << "The first result is " << z;
  cout << "The second result is " << q;
}


In [14]:
def is_function_call(funcdecl, c):
    defn = c.get_definition()
    return (defn is not None) and (defn == funcdecl)

def fully_qualified(c):
    res = c.spelling
    c = c.semantic_parent
    while c.kind != CursorKind.TRANSLATION_UNIT:
        res = c.spelling + '::' + res
        c = c.semantic_parent
    return res

## Using just the libclang API

In [15]:
def find_funcs_and_calls(tu):
    filename = tu.cursor.spelling
    calls = []
    funcs = []
    for c in tu.cursor.walk_preorder():
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
        elif c.kind == CursorKind.CALL_EXPR:
            calls.append(c)
        elif c.kind == CursorKind.FUNCTION_DECL:
            funcs.append(c)
    return funcs, calls

idx = Index.create()
args =  '-x c++ --std=c++11'.split()
tu = idx.parse('tmp.cpp', args=args)
funcs, calls = find_funcs_and_calls(tu)
for f in funcs:
    print fully_qualified(f)
    for c in calls:
        if is_function_call(f, c):
            print '-', c.location
    print


impl::addition
- <SourceLocation file 'tmp.cpp', line 10, column 9>

impl::f

addition
- <SourceLocation file 'tmp.cpp', line 22, column 7>
- <SourceLocation file 'tmp.cpp', line 23, column 7>

main



## Using glud

In this case, it's harder to read the glud code than the libclang code - however I can certainly write code like this much faster.

In [16]:
import glud
from glud.composition import *
from glud.predicates import *

tu = glud.parse('tmp.cpp', args='-x c++ --std=c++11'.split())
is_in_tu_main = is_in_file(tu.cursor.spelling)
is_func = all_fn([is_in_tu_main, is_func])
is_call = all_fn([is_in_tu_main, is_kind(CursorKind.CALL_EXPR)])
funcs = glud.walk(is_func, tu.cursor)
calls = list(glud.walk(is_call, tu.cursor)()

for f in funcs:
    print fully_qualified(f)
    for c in calls:
        if is_function_call(f, c):
            print '-', c.location
    print

impl::addition
- <SourceLocation file 'tmp.cpp', line 10, column 9>

impl::f

addition
- <SourceLocation file 'tmp.cpp', line 22, column 7>
- <SourceLocation file 'tmp.cpp', line 23, column 7>

main

